# Crear Target

Instalamos / actualizamos librerías

In [1]:
##%%bash 
# ! pip install duckdb
# ! pip install jupysql
# ! pip install duckdb-engine
# ! pip install pandas

In [4]:
# ! pip install --upgrade duckdb
# ! pip install --upgrade jupysql
# ! pip install --upgrade duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/dmeyf.db

Cargamos el archivo `.csv` a una tabla:

In [2]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/datasets/competencia_02_crudo.csv.gz')


,Success


Observamos registros para un cliente particular:

In [3]:
%%sql
SELECT 
    * 
FROM competencia_02
WHERE numero_de_cliente = 29215143

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29215143,201901,1,0,1,55,257,18365.44,108081.78,12409.78,...,0.0,8,0.00,-68889.96,972.95,7674,26329.35,16,0,2064.48
1,29215143,201902,1,0,1,55,258,5537.20,111874.25,968.74,...,0.0,8,0.00,-37967.32,1.00,7702,31997.89,15,0,2604.06
2,29215143,201903,1,0,1,55,259,10687.02,115257.44,1774.26,...,0.0,11,62738.10,-33935.55,96.12,7733,50367.38,22,0,1677.39
3,29215143,201904,1,0,1,55,260,9781.35,115405.24,2811.05,...,0.0,6,0.00,-62766.39,56.57,7763,34444.87,11,0,2662.71
4,29215143,201905,1,0,1,55,261,0.00,0.00,0.00,...,0.0,9,0.00,-49656.19,1.00,7794,34153.79,18,0,2604.06
5,29215143,201906,1,0,1,55,262,17463.89,128846.58,4773.25,...,0.0,12,30035.79,-48918.22,1.00,7824,30828.92,12,0,1818.15
6,29215143,201907,1,0,1,55,263,10091.09,133807.98,4458.39,...,0.0,7,0.00,-30035.28,1.00,7855,32813.97,18,0,2416.38
7,29215143,201908,1,0,1,55,264,8363.86,138507.59,3394.95,...,0.0,10,0.00,-34593.16,1.00,7886,39866.51,13,0,2498.49
8,29215143,201909,1,0,1,55,265,1385.99,135662.45,1854.24,...,0.0,12,55849.21,-42180.50,24.90,7916,54427.72,18,0,3284.40
9,29215143,201910,1,0,1,55,266,0.00,0.00,0.00,...,0.0,8,0.00,-55824.60,44.73,7947,51748.88,15,0,2826.93


Basado en sugerencia del profesor se construye target, se incluye lógica para generar la clase_ternaria:

In [4]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , case 
        when (mes_0 = 1) AND (mes_1 = 0) AND  (mes_2 = 0) THEN 'BAJA+1'
        when (mes_0 = 1) AND (mes_1 = 0) AND  (mes_2 = 1) THEN 'CONTINUA'
        when (mes_0 = 1) AND (mes_1 = 1) AND  (mes_2 = 0) THEN 'BAJA+2'
        when (mes_0 = 1) AND (mes_1 = 1) AND  (mes_2 = 1) THEN 'CONTINUA'
        else NULL end as clase_ternaria 
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
    foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


Algunas verificaciones:

In [31]:
%%sql
SELECT 
    * 
FROM targets
WHERE (mes_0 = 1) AND (mes_1 = 0) AND  (mes_2 = 1)

,numero_de_cliente,foto_mes,mes_0,mes_1,mes_2,clase_ternaria
0,30460466,201911,1,0,1,CONTINUA
1,100084897,201912,1,0,1,CONTINUA
2,153254472,202008,1,0,1,CONTINUA
3,109321500,202011,1,0,1,CONTINUA
4,127524491,202005,1,0,1,CONTINUA
5,33496786,201908,1,0,1,CONTINUA
6,76577049,202004,1,0,1,CONTINUA
7,114811302,201902,1,0,1,CONTINUA
8,105229225,202004,1,0,1,CONTINUA
9,116673221,201901,1,0,1,CONTINUA


In [30]:
%%sql
SELECT 
    * 
FROM targets
WHERE numero_de_cliente=30147596

,numero_de_cliente,foto_mes,mes_0,mes_1,mes_2,clase_ternaria
0,30147596,201901,1,1,1,CONTINUA
1,30147596,201902,1,1,1,CONTINUA
2,30147596,201903,1,1,1,CONTINUA
3,30147596,201904,1,1,1,CONTINUA
4,30147596,201905,1,1,1,CONTINUA
5,30147596,201906,1,1,1,CONTINUA
6,30147596,201907,1,1,1,CONTINUA
7,30147596,201908,1,1,1,CONTINUA
8,30147596,201909,1,1,1,CONTINUA
9,30147596,201910,1,1,1,CONTINUA


In [5]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [6]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

,Success


Cuento cuántos casos tengo con BAJA+2 y cuántas filas tiene mi periodo a predecir 202107:

In [ ]:
%%sql
SELECT 
    foto_mes, 
    clase_ternaria, 
    count(*) as cant_bajas
FROM competencia_02
WHERE clase_ternaria = 'BAJA+2'
GROUP BY 
    foto_mes, 
    clase_ternaria
ORDER BY foto_mes DESC

In [13]:
%%sql
SELECT 
    foto_mes, 
    clase_ternaria, 
    count(*) as cant_bajas
FROM competencia_02
WHERE clase_ternaria IS NULL 
GROUP BY 
    foto_mes, 
    clase_ternaria
ORDER BY foto_mes DESC

,foto_mes,clase_ternaria,cant_bajas
0,202107,NaN,164682
1,202106,NaN,164420


Guardamos el dataset final:

In [14]:
%%sql
copy competencia_02 to 'C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)

,Success
